[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2-70b-chat-agent.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/llama-2-70b-chat-agent.ipynb)

# LLama-2 Conversational Agent in Petals and LangChain

In [ ]:
%pip install --no-cache-dir -qU petals xformers accelerate langchain transformers[torch]

### Default Prompt Template

In [1]:
# REF: https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat/blob/main/model.py#L24
DEFAULT_SYSTEM_PROMPT = "You are a helpful assistant.\nYou will try to answer user questions, but don't make up the answer if you don't have the answer.\nYou will complete tasks by following user instructions."
def get_prompt(message: str, chat_history: list[tuple[str, str]] = [],
               system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    texts = [f'[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']
    for user_input, response in chat_history:
        texts.append(f'{user_input.strip()} [/INST] {response.strip()} </s><s> [INST] ')
    texts.append(f'{message.strip()} [/INST]')
    return ''.join(texts)

### Setup Environment Variables

In [2]:
import os

os.environ["HUGGINGFACE_API_KEY"] = "hf_XXXX"
os.environ["TRANSFORMERS_CACHE"] = "This is optional. I set the cache directory of my external HDD."            

### Initiate the LLM and run a prediction

In [3]:
import torch
from custom.llms.petals import Petals

llm = Petals(
    temperature=0.01,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    torch_dtype=torch.float32
)

/media/limcheekin/My Passport/langchain-playground/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/limcheekin/My Passport/langchain-playground/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/media/limcheekin/My Passport/langchain-playground/venv/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Jul 26 16:18:57.766 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Jul 26 16:18:57.767 [INFO] Using DHT prefix: Llama-2-70b-chat

In [4]:
llm(prompt=get_prompt("Explain to me the difference between nuclear fission and fusion."))

Jul 26 16:19:18.534 [INFO] Route found: 0:40 via …uCst2v => 40:80 via …o6L7CF


"<s> [INST] <<SYS>>\nYou are a helpful assistant.\nYou will try to answer user questions, but don't make up the answer if you don't have the answer.\nYou will complete tasks by following user instructions.\n<</SYS>>\n\nExplain to me the difference between nuclear fission and fusion. [/INST]  Sure, I'd be happy to help!\n\nNuclear fission is a process in which an atomic nucleus splits into two or more smaller nuclei, releasing a large amount of energy in the process. This occurs when an atom's nucleus is bombarded with a high-energy particle, such as a neutron. The resulting nuclei are typically smaller and lighter than the original nucleus, and the excess energy is released as radiation. Nuclear fission is the process used in nuclear power plants to generate electricity.\n\nNuclear fusion, on the other hand, is the process in which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases a large amount of energy, but it is different from fission i

### Initializing an Conversational Agent

Getting a conversational agent to work with open source models is incredibly hard. However, with Llama 2 70B it is now possible. Let's see how we can get it running!

We first need to initialize the agent. Conversational agents require several things such as conversational `memory`, access to `tools`, and an `llm` (which we have already initialized).

In [5]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools

memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)
tools = load_tools(["llm-math"], llm=llm)

In [6]:
from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish

class OutputParser(AgentOutputParser):
    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> AgentAction | AgentFinish:
        try:
            # this will work IF the text is a valid JSON with action and action_input
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                # this means the agent is finished so we call AgentFinish
                return AgentFinish({"output": action_input}, text)
            else:
                # otherwise the agent wants to use an action, so we call AgentAction
                return AgentAction(action, action_input, text)
        except Exception:
            # sometimes the agent will return a string that is not a valid JSON
            # often this happens when the agent is finished
            # so we just return the text as the output
            return AgentFinish({"output": text}, text)

    @property
    def _type(self) -> str:
        return "conversational_chat"

# initialize output parser for agent
parser = OutputParser()

In [7]:
from langchain.agents import initialize_agent

# initialize agent
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    handle_parsing_errors=True,
    memory=memory,
    # agent_kwargs={"output_parser": parser}
)

In [8]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

We need to add special tokens used to signify the beginning and end of instructions, and the beginning and end of system messages. These are described in the Llama-2 model cards on Hugging Face.

In [9]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

In [10]:
sys_msg = B_SYS + """Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {{"action": "Calculator",
      "action_input": "sqrt(4)"}}
    ```

Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}}
```
User: I'm great, what is the square root of 4?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "sqrt(4)"}}
```
User: 2.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 2!"}}
```
User: Thanks could you tell me what 4 to the power of 2 is?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "4**2"}}
```
User: 16.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 16!"}}
```

Here is the latest conversation between Assistant and User.""" + E_SYS
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
agent.agent.llm_chain.prompt = new_prompt

In the Llama 2 paper they mentioned that it was difficult to keep Llama 2 chat models following instructions over multiple interactions. One way they found that improves this is by inserting a reminder of the instructions to each user query. We do that here:

In [11]:
instruction = B_INST + " Respond to the following in JSON with 'action' and 'action_input' values " + E_INST
human_msg = instruction + "\nUser: {input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

In [12]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='<<SYS>>\nAssistant is a expert JSON builder designed to assist with a wide range of tasks.\n\nAssistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.\n\nAll of Assistant\'s communication is performed using this JSON format.\n\nAssistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:\n\n- "Calculator": Useful for when you need to answer questions about math.\n  - To use the calculator tool, Assistant should write like so:\n    ```json\n    {{"action": "Calculator",\n      "action_input": "sqrt(4)"}}\n    ```\n\nHere are some previous conversations between 

Now we can begin asking questions...

In [13]:
agent("hey how are you today?")

Jul 26 16:27:04.948 [INFO] Route found: 0:40 via …uCst2v => 40:80 via …o6L7CF




> Entering new AgentExecutor chain...


Jul 26 16:37:44.483 [INFO] Route found: 0:40 via …uCst2v => 40:80 via …o6L7CF


<s> System: <<SYS>>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {"action": "Calculator",
      "action_input": "sqrt(4)"}
    ```

Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}
```
User: I'm great, what is the square root of 4?
Assistant: ```json
{"action": "Calculator",
 "action_input": "

ValueError: unknown format from LLM: <s> Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...

In [ ]:
agent("what is 4 to the power of 2.1?")

In [ ]:
agent("can you multiply that by 3?")